In [6]:
import os
import sys
import pandas as pd
import requests
import re
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
import json

In [17]:
browser = Chrome()
delay = 3
base_url = 'https://cafe.naver.com/thankmall'

In [18]:
browser.get(base_url)
browser.implicitly_wait(delay)

In [19]:
browser.find_element_by_css_selector('span.gnb_txt').click()
browser.implicitly_wait(delay)

## 네이버 로그인 보안 강화로 코드로 접속 불가, 직접 쳐야한다.
#### browser.find_element_by_name('id').send_keys(naver_id)
#### browser.find_element_by_name('pw').send_keys(naver_pw)
#### browser.find_element_by_css_selector('input.btn_global').send_keys(Keys.ENTER)

In [20]:
cafe_menu = browser.find_element_by_css_selector('div#cafe-menu')
browser.implicitly_wait(delay)

In [21]:
menu_list = cafe_menu.find_elements_by_tag_name('a')

In [22]:
menu_source = pd.DataFrame({'name' : [], 'source' : []})
name = []
source = []

for menu in menu_list :
    menu_name = menu.text
    if '전체글' in menu_name : # 원하는 검색어를 집어넣어야 한다.
        insert_menu = pd.DataFrame({'name' : [menu_name], 'source' : [menu]})
        menu_source = menu_source.append(insert_menu)

In [23]:
menu_source.index = range(len(menu_source))

In [24]:
menu_source

,name,source
0,전체글보기,<selenium.webdriver.remote.webelement.WebEleme...


In [25]:
# 원하는 카테고리 클릭을 위해서 인덱싱을 진행하는데, 앞에부분만 바꾸면된다.
menu_source.iloc[0,1].send_keys(Keys.ENTER)
                                            
browser.implicitly_wait(delay)

In [26]:
i = 1
post_list = pd.DataFrame({'name' : [],
                         'user' : [],
                         'time' : [],
                         'link' : []})
while True :
    try :
        browser.switch_to_frame(browser.find_element_by_name('cafe_main'))
        page = browser.find_element_by_css_selector('div.prev-next')
        page_num = page.find_elements_by_tag_name('a')
    except :
        page = browser.find_element_by_css_selector('div.prev-next')
        page_num = page.find_elements_by_tag_name('a')        
    
    for num in range(len(page_num)) :           
            try :  
                browser.switch_to_frame(browser.find_element_by_name('cafe_main'))
                page = browser.find_element_by_css_selector('div.prev-next')
                page_num = page.find_elements_by_tag_name('a')
                if '다음' in page_num[num].text :
                    page_num[num].send_keys(Keys.ENTER)
                    break
                elif '이전' in page_num[num].text :
                    pass
                else :
                    page_num[num].send_keys(Keys.ENTER)
            except :
                page = browser.find_element_by_css_selector('div.prev-next')
                page_num = page.find_elements_by_tag_name('a')
                if '다음' in page_num[num].text :
                    page_num[num].send_keys(Keys.ENTER)
                    break        
                elif '이전' in page_num[num].text :
                    pass
                else :
                    page_num[num].send_keys(Keys.ENTER)
                
            source = browser.page_source
            html = BeautifulSoup(source, 'html.parser')
            tbody = html.find_all('div', {'class' : 'article-board m-tcol-c'})
            post_name = tbody[1].find_all('a', {'class' :'article'})
            post_user = tbody[1].find_all('a', {'class' : 'm-tcol-c'})
            post_time = tbody[1].find_all('td', {'class' : 'td_date'})

            name = []
            user = []
            time = []
            link = []

            for n in post_name :
                n = n.text.replace('\n','')
                name.append(re.sub(r' {2,}' , ' ', n))
                
            for u in post_user :
                user.append(u.text)

            for t in post_time :
                time.append(re.sub('[^0-9\.\:]', '', t.text))
                
            for l in range(len(post_name)) :
                post_link = base_url + post_name[l]['href']
                link.append(post_link)

            posting = pd.DataFrame({'name' : name,
                                    'user' : user,
                                    'time' : time,
                                   'link' : link})

            post_list = post_list.append(posting)

    if len(page_num) < 11 :
        break
    
post_list = post_list.drop_duplicates(subset='name', keep='last')
post_list.index = range(len(post_list))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.frame instead
  


IndexError: list index out of range

In [27]:
post_list = post_list.drop_duplicates(subset='name', keep='last')
post_list.index = range(len(post_list))

post_list

,name,user,time,link
0,[ 퀄리티 최상급 ] [ 아동복 국산 브랜드 >> 땡처리 합니다 ],멕가이버,15:41,https://cafe.naver.com/thankmall/ArticleRead.n...
1,[완사입/파샬]퀄리티 좋은 봄 쟈가드 롱 가디건 2컬러 - 200여장 8000원,가비야IPKOREA,14:19,https://cafe.naver.com/thankmall/ArticleRead.n...
2,[완사입] 언발롱셔츠 - 5장 5000원,가비야IPKOREA,2019.02.07.,https://cafe.naver.com/thankmall/ArticleRead.n...
3,[완사입/파샬] 미시간 쭈리 원피스 2컬러 - 34장 4000원,가비야IPKOREA,2019.02.07.,https://cafe.naver.com/thankmall/ArticleRead.n...
4,행복한 설명절되세요~~~~~~^^,작은별,2019.02.03.,https://cafe.naver.com/thankmall/ArticleRead.n...
5,쿠첸생산 수출IH다이야몬드철정압력밥솥 10인용,새영,2019.02.03.,https://cafe.naver.com/thankmall/ArticleRead.n...
6,차렵이불.패드입니다,물고기사랑,2019.02.03.,https://cafe.naver.com/thankmall/ArticleRead.n...
7,새해 복 많이 받으세요~~~,phss0802,2019.02.02.,https://cafe.naver.com/thankmall/ArticleRead.n...
8,아동 봄 상하세트 110셋,마마미,2019.02.02.,https://cafe.naver.com/thankmall/ArticleRead.n...
9,여성 봄니트 대봉 3만원,지구정복,2019.02.02.,https://cafe.naver.com/thankmall/ArticleRead.n...


In [28]:
post_list.to_csv(os.getcwd() + "/" + "cafe_thankmall" + ".csv", mode = 'w', header = True, index = None, encoding = 'utf-8-sig')

In [29]:
post_list['user'].value_counts()

현민유통김이사             879
남양주걸크러쉬             627
가비야IPKOREA          445
물고기사랑               328
최건샵                 289
옷파는김사장              281
이쁜건다                276
작은별                 259
마유리                 217
향기로운만남              190
썬스페이스               157
행복이1                147
쇼미더머니               134
이싸                  128
범진통상                120
선장                  112
알제이샵                 96
더미더미                 94
토끼                   93
대박포촤                 91
소량대량                 79
소다아동복                77
울루맘                  76
잡화상                  75
비엠의류                 74
리더스코리아               70
이벤트                  68
뿌요뿌요                 65
도도맘                  61
린꽃                   61
                   ... 
무지개창고                 1
수산 is1877 kapsan      1
진진진진                  1
yruiyqq               1
렝가                    1
쏘롱이                   1
왕눈이삼촌                 1
대박장터                  1
스토리제이                 1
굳센넘                   1
잇스 ITS          

In [30]:
post_list['user'].value_counts().to_csv(os.getcwd() + "/" + "cafe_thankmall_userlist" + ".csv", mode = 'w', header = True, encoding = 'utf-8-sig')

# 땡몰 여성, 남성 의류 덤핑 크롤러

In [10]:
delay = 3
base_url = 'https://cafe.naver.com/thankmall'

In [11]:
browser = Chrome()
browser.maximize_window()
time.sleep(2)
browser.get(base_url)
browser.implicitly_wait(delay)

In [12]:
body = browser.find_element_by_tag_name('body')
num_page_down = 2
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    num_page_down -= 1

In [13]:
#제어 환경에 보여야 함
browser.find_elements_by_xpath('//*[@id = "group95"]/li[1]/a')[0].click()

In [14]:
dumping_data = pd.DataFrame({'post_title':[],
                           'owner':[],
                           'date':[],
                           'url':[]})
#ifrme to html 코드 굉장히 중요함
browser.switch_to_frame(browser.find_element_by_name('cafe_main'))
for i in range(1,3):
    a=i
    for j in range(1,12):#그래야 1~11까지의 범위가 됨
        try:
            if a==1:
                body = browser.find_element_by_tag_name('body')
                num_page_down = 1
                while num_page_down:
                    body.send_keys(Keys.PAGE_DOWN)
                    time.sleep(1)
                    num_page_down -= 1
                browser.find_elements_by_xpath('//*[@class = "prev-next"]/a['+ str(j) +']')[0].click()

                    
                html0 = browser.page_source    
                html= BeautifulSoup(html0,'html.parser')
                post0 = html.find_all('div',{'class':'article-board m-tcol-c'})[1]
                post1 = post0.find('tbody')
                
                post_title0 = post1.find_all('a',{'class':'article'})
                post_owner0 = post1.find_all('td',{'class':'p-nick'})
                post_date0 = post1.find_all('td',{'class':'td_date'})
                
                insert_data = pd.DataFrame({'post_title':[],
                                           'owner':[],
                                           'date':[],
                                           'url':[]})
                for post in range(len(post_title0)):
                    post_title = "".join(post_title0[post].text.split())
                    post_url = 'http://cafe.naver.com'+ post_title0[post]['href']
                    post_owner = post_owner0[post].find('a').text
                    post_date = post_date0[post].text
                    
                    insert_data2 =  pd.DataFrame({'post_title':[post_title],
                                           'owner':[post_owner],
                                           'date':[post_date],
                                           'url':[post_url]})
                    insert_data = insert_data.append(insert_data2)
                    
                dumping_data = dumping_data.append(insert_data)
                
            else:
                if j == 1:
                    pass
                else:
                    body = browser.find_element_by_tag_name('body')
                    num_page_down = 1
                    while num_page_down:
                        body.send_keys(Keys.PAGE_DOWN)
                        time.sleep(1)
                        num_page_down -= 1
                    browser.find_elements_by_xpath('//*[@class = "prev-next"]/a['+ str(j+1) +']')[0].click()

                
                html0 = browser.page_source    
                html= BeautifulSoup(html0,'html.parser')
                post0 = html.find_all('div',{'class':'article-board m-tcol-c'})[1]
                post1 = post0.find('tbody')
                
                post_title0 = post1.find_all('a',{'class':'article'})
                post_owner0 = post1.find_all('td',{'class':'p-nick'})
                post_date0 = post1.find_all('td',{'class':'td_date'})
                
                insert_data = pd.DataFrame({'post_title':[],
                                           'owner':[],
                                           'date':[],
                                           'url':[]})
                for post in range(len(post_title0)):
                    post_title = "".join(post_title0[post].text.split())
                    post_url = 'http://cafe.naver.com'+ post_title0[post]['href']
                    post_owner = post_owner0[post].find('a').text
                    post_date = post_date0[post].text
                    
                    insert_data2 =  pd.DataFrame({'post_title':[post_title],
                                           'owner':[post_owner],
                                           'date':[post_date],
                                           'url':[post_url]})
                    insert_data = insert_data.append(insert_data2)
                    
                dumping_data = dumping_data.append(insert_data)
        except:
            pass

In [16]:
dumping_data.index = range(len(dumping_data))

In [ ]:
# 긁는 도중 삭제한 게시글이 있어서 다시 진행해야ㅑ함

In [29]:
dumping_data2 = dumping_data.iloc[-7]

# 본문 긁어오기

In [22]:
base_url2  = dumping_data['url'].iloc[0]

In [23]:
browser = Chrome()
browser.maximize_window()
time.sleep(2)
browser.get(base_url2)

In [26]:
post_main_data = pd.DataFrame({'post_title':[],
                           'owner':[],
                           'date':[],
                           'url':[],
                           'main_text':[]})
for i in range(0,101):
    try:
        base_url2 = dumping_data['url'].iloc[i]
        browser.get(base_url2)
        browser.implicitly_wait(delay)
        #새창을 켜게 된다면 로그인을 새로해야 하는데 네이버 측에서 자동 제어 로그인을 막아놨기 때문에
        #먼저 로그인을 한 후 진행해야 함

        post_title = dumping_data['post_title'].iloc[i]
        owner = dumping_data['owner'].iloc[i]
        date = dumping_data['date'].iloc[i]

        browser.switch_to_frame(browser.find_element_by_name('cafe_main'))
        html_all0 = browser.page_source
        html_all = BeautifulSoup(html_all0,'html.parser')

        post_text = str(html_all.find('div',{'class':'tbody m-tcol-c'}).get_text())
        post_text2 = re.sub('\xa0',' ',post_text)

        if 'img' in str(html_all.find('div',{'class':'tbody m-tcol-c'})):
            post_img = html_all.find('div',{'class':'tbody m-tcol-c'}).find_all('img')
            #src를 어떻게 코드상에 최적화 시킬지
            for i in range(len(post_img)):
                img_0 = post_img[i]['src']
                img_source = '<img src = '+ '"'+img_0+ '"'+' >\n'
                post_text3 = post_text2 + img_source
            main_text = post_text3
        else:
            main_text = post_text2


        url = base_url2

        insert_data = pd.DataFrame({'post_title':[post_title],
                               'owner':[owner],
                               'date':[date],
                               'url':[url],
                               'main_text':[main_text]})
        post_main_data = post_main_data.append(insert_data)
    except:
        pass
post_main_data.index = range(len(post_main_data))   

In [31]:
post_main_data

,post_title,owner,date,url,main_text
0,2019년홈쇼핑대박상품여성의류최저가공급,행복이보인다,15:43,http://cafe.naver.com/ArticleRead.nhn?clubid=2...,"\n \n<img src = ""https://cafeptthumb-phinf.pst..."
1,"비비드,웜,네온컬러셔츠2000원",덤핑구,15:08,http://cafe.naver.com/ArticleRead.nhn?clubid=2...,"\n#의류 #덤핑 #국내생산 #동대문국내생산 동대문 셔츠 판매합니다.컬러는 비비드,..."
2,남성피그먼트워싱밴딩팬츠정리합니다.,스타일조아,2019.04.11.,http://cafe.naver.com/ArticleRead.nhn?clubid=2...,\n피그먼트 워싱 돌린 밴딩 팬츠입니다.프리사이즈로 남성 허리 34까지는 무난하게 ...
3,홈쇼핑요가복5종1000원에100셋트씩판매합니다.,썬스페이스,2019.04.11.,http://cafe.naver.com/ArticleRead.nhn?clubid=2...,"\n안녕하세요?일산창고 ""땡스토리 ""입니다.홈쇼핑 요가복5종1000원에100셋트씩판..."
4,니트롱가디건입니다,하스모나,2019.04.11.,http://cafe.naver.com/ArticleRead.nhn?clubid=2...,"\n니트 롱 가디건 입니다국산 제품입니다수량이 많이 없습니다장당 5,000원010 ..."
5,봄.여름원피스.블라우스특가(초이스가능),하스모나,2019.04.11.,http://cafe.naver.com/ArticleRead.nhn?clubid=2...,\n원피스.블라우스 동대문 밤시장 제품입니다.여러가지 스타일로 만여장 입니다더 많은...
6,여성브랜드팬티..600원,물고기사랑,2019.04.10.,http://cafe.naver.com/ArticleRead.nhn?clubid=2...,\n코튼클럽.BYC 제품입니다 사이즈90호만...인견.순먼.라이크라 ...010.9...
